### Постановка задачи
Подготовим данные для построения модели: получим, объединим, оптимизируем и обогатим данные.

Сохраним готовые данные в нескольких форматах: CSV, HDF5

Данные:
* http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz
Соревнование: https://www.kaggle.com/c/ashrae-energy-prediction/

© ITtensive, 2020

### Подключение библиотек

In [37]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import numpy as np
from scipy.interpolate import interp1d
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import os

### Загрузка данных, отсечение 20 зданий, объединение и оптимизация

In [38]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        elif col == "timestamp":
            df[col] = pd.to_datetime(df[col])
        elif str(col_type)[:8] != "datetime":
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

In [39]:
buildings = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz")
weather = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz")
weather = weather[weather["site_id"] == 0]
energy = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz")
energy = energy[energy["building_id"] < 20]
energy = pd.merge(left=energy, right=buildings, how="left",
                   left_on="building_id", right_on="building_id")

In [40]:
print ("Строения: ", buildings.memory_usage().sum() / 1024**2, "Мб")
print ("Погода: ", weather.memory_usage().sum() / 1024**2, "Мб")
print ("Энергопотребление: ", energy.memory_usage().sum() / 1024**2, "Мб")

Строения:  0.0664520263671875 Мб
Погода:  0.670166015625 Мб
Энергопотребление:  13.4033203125 Мб


In [41]:
del buildings

### Интерполяция значений

In [42]:
weather["precip_depth_1_hr"] = weather["precip_depth_1_hr"].apply(lambda x:x if x>0 else 0)
interpolate_columns = ["air_temperature", "dew_temperature",
                       "cloud_coverage", "wind_speed", "wind_direction",
                       "precip_depth_1_hr", "sea_level_pressure"]
for col in interpolate_columns:
    weather[col] = weather[col].interpolate(limit_direction='both',
                            kind='cubic')
print(weather["precip_depth_1_hr"])

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
8779    0.0
8780    0.0
8781    0.0
8782    0.0
8783    0.0
Name: precip_depth_1_hr, Length: 8784, dtype: float64


### Обогащение данных: погода

In [43]:
weather["air_temperature_diff1"] = weather["air_temperature"].diff()
weather.at[0, "air_temperature_diff1"] = weather.at[1, "air_temperature_diff1"]
weather["air_temperature_diff2"] = weather["air_temperature_diff1"].diff()
weather.at[0, "air_temperature_diff2"] = weather.at[1, "air_temperature_diff2"]
print(weather["air_temperature_diff1"])
print(weather.at[0, "air_temperature_diff1"])

print(weather["air_temperature_diff2"])
print(weather.at[0, "air_temperature_diff2"])

0      -0.6
1      -0.6
2      -1.6
3      -1.7
4      -1.1
       ... 
8779    1.1
8780    0.5
8781    0.0
8782   -0.5
8783   -2.2
Name: air_temperature_diff1, Length: 8784, dtype: float64
-0.6000000000000014
0       0.0
1       0.0
2      -1.0
3      -0.1
4       0.6
       ... 
8779   -0.6
8780   -0.6
8781   -0.5
8782   -0.5
8783   -1.7
Name: air_temperature_diff2, Length: 8784, dtype: float64
0.0


### Объединение погодных данных

In [44]:
energy = energy.set_index(["timestamp", "site_id"])
weather = weather.set_index(["timestamp", "site_id"])
energy = pd.merge(left=energy, right=weather, how="left",
                  left_index=True, right_index=True)
energy.reset_index(inplace=True)
energy = energy.drop(columns=["meter", "site_id", "year_built",
                              "square_feet", "floor_count"], axis=1)
energy = reduce_mem_usage(energy)
del weather
print (energy.info())

Потребление памяти меньше на 12.4 Мб (минус 71.2 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175680 entries, 0 to 175679
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   timestamp              175680 non-null  datetime64[ns]
 1   building_id            175680 non-null  int8          
 2   meter_reading          175680 non-null  float16       
 3   primary_use            175680 non-null  category      
 4   air_temperature        175680 non-null  float16       
 5   cloud_coverage         175680 non-null  float16       
 6   dew_temperature        175680 non-null  float16       
 7   precip_depth_1_hr      175680 non-null  float16       
 8   sea_level_pressure     175680 non-null  float16       
 9   wind_direction         175680 non-null  float16       
 10  wind_speed             175680 non-null  float16       
 11  air_temperature_diff1  175680 non-null  float16     

### Обогащение данных: дата

In [45]:
energy["hour"] = energy["timestamp"].dt.hour.astype("int8")
energy["weekday"] = energy["timestamp"].dt.weekday.astype("int8")
energy["week"] = energy["timestamp"].dt.week.astype("int8")
energy["month"] = energy["timestamp"].dt.month.astype("int8")
energy["date"] = pd.to_datetime(energy["timestamp"].dt.date)
dates_range = pd.date_range(start='2015-12-31', end='2017-01-01')
us_holidays = calendar().holidays(start=dates_range.min(),
                                  end=dates_range.max())
energy['is_holiday'] = energy['date'].isin(us_holidays).astype("int8")
for weekday in range(0,7):
    energy['is_wday' + str(weekday)] = energy['weekday'].isin([weekday]).astype("int8")
for week in range(1,54):
    energy['is_w' + str(week)] = energy['week'].isin([week]).astype("int8")
for month in range(1,13):
    energy['is_m' + str(month)] = energy['month'].isin([month]).astype("int8")

C:\Users\NRKP\AppData\Local\Temp\ipykernel_892\3950203926.py:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  energy["week"] = energy["timestamp"].dt.week.astype("int8")


### Логарифмирование данных
z = A * x + B * y -> log z = A * x + B * y => z = e^Ax * e^By => z = a^x * b^y

In [46]:
energy["meter_reading_log"] = np.log(energy["meter_reading"] + 1)

### Экспорт данных в CSV и HDF5

In [47]:
print (energy.info())
energy.to_csv("energy.0-20.ready.csv.gz", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175680 entries, 0 to 175679
Data columns (total 92 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   timestamp              175680 non-null  datetime64[ns]
 1   building_id            175680 non-null  int8          
 2   meter_reading          175680 non-null  float16       
 3   primary_use            175680 non-null  category      
 4   air_temperature        175680 non-null  float16       
 5   cloud_coverage         175680 non-null  float16       
 6   dew_temperature        175680 non-null  float16       
 7   precip_depth_1_hr      175680 non-null  float16       
 8   sea_level_pressure     175680 non-null  float16       
 9   wind_direction         175680 non-null  float16       
 10  wind_speed             175680 non-null  float16       
 11  air_temperature_diff1  175680 non-null  float16       
 12  air_temperature_diff2  175680 non-null  floa

In [48]:
energy = pd.read_csv("energy.0-20.ready.csv.gz")
print (energy.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175680 entries, 0 to 175679
Data columns (total 92 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   timestamp              175680 non-null  object 
 1   building_id            175680 non-null  int64  
 2   meter_reading          175680 non-null  float64
 3   primary_use            175680 non-null  object 
 4   air_temperature        175680 non-null  float64
 5   cloud_coverage         175680 non-null  float64
 6   dew_temperature        175680 non-null  float64
 7   precip_depth_1_hr      175680 non-null  float64
 8   sea_level_pressure     175680 non-null  float64
 9   wind_direction         175680 non-null  float64
 10  wind_speed             175680 non-null  float64
 11  air_temperature_diff1  175680 non-null  float64
 12  air_temperature_diff2  175680 non-null  float64
 13  hour                   175680 non-null  int64  
 14  weekday                175680 non-nu

### Экспорт данных в HDF5
HDF5: / ->
* Группа (+ метаданные)
 * Набор данных
 * ...

In [49]:
energy = reduce_mem_usage(energy)
energy.to_hdf('energy.0-20.ready.h5', "energy", format='table', mode="w")
print ("CSV:", os.path.getsize(os.getcwd() + '\energy.0-20.ready.csv.gz'))
print ("HDF5:", os.path.getsize(os.getcwd() + '\energy.0-20.ready.h5'))

Потребление памяти меньше на 104.7 Мб (минус 84.9 %)
CSV: 1529689
HDF5: 21300602


C:\anaconda3\lib\site-packages\tables\file.py:426: UserWarning: a closed node found in the registry: ``/energy/meta/values_block_1/meta/_i_table``
  warnings.warn("a closed node found in the registry: "


In [50]:
energy = pd.read_hdf('energy.0-20.ready.h5', "energy")
print (energy.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175680 entries, 0 to 175679
Data columns (total 92 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   timestamp              175680 non-null  datetime64[ns]
 1   building_id            175680 non-null  int8          
 2   meter_reading          175680 non-null  float16       
 3   primary_use            175680 non-null  category      
 4   air_temperature        175680 non-null  float16       
 5   cloud_coverage         175680 non-null  float16       
 6   dew_temperature        175680 non-null  float16       
 7   precip_depth_1_hr      175680 non-null  float16       
 8   sea_level_pressure     175680 non-null  float16       
 9   wind_direction         175680 non-null  float16       
 10  wind_speed             175680 non-null  float16       
 11  air_temperature_diff1  175680 non-null  float16       
 12  air_temperature_diff2  175680 non-null  floa

### Разделение данных и экспорт в HDF5

In [51]:
energy_train, energy_test = train_test_split(energy[energy["meter_reading"]>0], test_size=0.2)
print (energy_train.head())

                 timestamp  building_id  meter_reading primary_use  \
69781  2016-05-25 09:00:00            1       125.8750   Education   
135389 2016-10-09 01:00:00            9       123.6875      Office   
121302 2016-09-09 17:00:00            2        49.5625   Education   
145637 2016-10-30 09:00:00           17       139.5000      Office   
120137 2016-09-07 06:00:00           17       121.2500      Office   

        air_temperature  cloud_coverage  dew_temperature  precip_depth_1_hr  \
69781          21.09375        2.666016        17.203125                0.0   
135389         26.09375        5.000000        21.703125                0.0   
121302         31.09375        5.500000        23.296875                0.0   
145637         20.00000        5.601562        18.296875                0.0   
120137         25.00000        8.000000        21.703125                0.0   

        sea_level_pressure  wind_direction  ...  is_m4  is_m5  is_m6  is_m7  \
69781               1021.

In [52]:
pd.set_option('io.hdf.default_format','table')
store = pd.HDFStore('energy.0-20.ready.split.h5', mode="w")
store["energy_train"] = energy_train
store["energy_test"] = energy_test
store.put("metadata",
             pd.Series(["Набор обогащенных тестовых данных по 20 зданиям"]))
store.close()
print ("HDF5:", os.path.getsize(os.getcwd() + '\energy.0-20.ready.split.h5'))

HDF5: 13687735


C:\anaconda3\lib\site-packages\tables\file.py:426: UserWarning: a closed node found in the registry: ``/energy_train/meta/values_block_4/meta/_i_table``
  warnings.warn("a closed node found in the registry: "


Для хранения атрибутов наборов данных также можно использовать

store.get_storer('energy_train').attrs.my_attr

### Чтение из HDF5

In [53]:
store = pd.HDFStore('energy.0-20.ready.split.h5')
energy_test = store.get("energy_test")[:]
energy_train = store.get("energy_train")[:]
metadata = store.get("metadata")[:]
store.close()
print (metadata[0])
print (energy_train.head())

Набор обогащенных тестовых данных по 20 зданиям
                 timestamp  building_id  meter_reading primary_use  \
69781  2016-05-25 09:00:00            1       125.8750   Education   
135389 2016-10-09 01:00:00            9       123.6875      Office   
121302 2016-09-09 17:00:00            2        49.5625   Education   
145637 2016-10-30 09:00:00           17       139.5000      Office   
120137 2016-09-07 06:00:00           17       121.2500      Office   

        air_temperature  cloud_coverage  dew_temperature  precip_depth_1_hr  \
69781          21.09375        2.666016        17.203125                0.0   
135389         26.09375        5.000000        21.703125                0.0   
121302         31.09375        5.500000        23.296875                0.0   
145637         20.00000        5.601562        18.296875                0.0   
120137         25.00000        8.000000        21.703125                0.0   

        sea_level_pressure  wind_direction  ...  is_m4  